Hi, to use this program, please click '**run all**' and skip all the way to '**VI) Results**'

Please ensure to input date and location when prompted, in order for the program to work.

---
##**Table of Contents**
> I) Calling carpark location data from data.gov.hk API

> II) Loading the json extracts into pandas dataframe

> III) Get Real-time Vacancy for each carpark

> IV) Join both datasets

> V) User location string input fuzzy match optimization

> VI) Results

#**I) Calling carpark location data from data.gov.hk API**

The API page [In English](https://data.gov.hk/en-data/dataset/hk-dpo-datagovhk1-carpark-info-vacancy/resource/01752c62-a6b6-4ddc-bf2d-25efccadc143)

Data dictionary [here](https://resource.data.one.gov.hk/opendata/carpark/Parking_Vacancy_Data_Specification.pdf).

In [119]:
import urllib.request, urllib.parse
import json
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bsoup
!pip install rapidfuzz
from rapidfuzz import process, fuzz

In [120]:
# get vacancy data

api_url = 'https://api.data.gov.hk/v1/carpark-info-vacancy'

param_vacancy = {'data': 'vacancy', 'vehicleTypes': 'privateCar'}
url_param_vacancy = api_url + '?' + urllib.parse.urlencode(param_vacancy)
raw_data_vacancy = urllib.request.urlopen(url_param_vacancy).read().decode()
js_result_vacancy = json.loads(raw_data_vacancy)

In [121]:
js_result_vacancy

{'results': [{'park_Id': '10',
   'privateCar': [{'vacancy_type': 'A',
     'vacancy': 105,
     'lastupdate': '2025-06-21 20:43:51'}]},
  {'park_Id': '12',
   'privateCar': [{'vacancy_type': 'A',
     'vacancyEV': 2,
     'vacancyDIS': 2,
     'vacancy': 26,
     'lastupdate': '2025-06-20 09:30:24'}],
   'LGV': [{'vacancy_type': 'A',
     'vacancy': 0,
     'lastupdate': '2025-06-20 09:30:24'}],
   'HGV': [{'vacancy_type': 'A',
     'vacancy': 0,
     'lastupdate': '2025-06-20 09:30:24'}],
   'motorCycle': [{'vacancy_type': 'A',
     'vacancy': 0,
     'lastupdate': '2025-06-20 09:30:24'}]},
  {'park_Id': '30',
   'privateCar': [{'vacancy_type': 'A',
     'vacancy': 11,
     'lastupdate': '2025-06-21 20:41:30'}]},
  {'park_Id': '31',
   'privateCar': [{'vacancy_type': 'A',
     'vacancy': 0,
     'lastupdate': '2025-06-21 19:28:00'}]},
  {'park_Id': '32',
   'privateCar': [{'vacancy_type': 'A',
     'vacancy': 0,
     'lastupdate': '2025-06-21 20:37:00'}]},
  {'park_Id': '37',
   'pri

In [122]:
param_info = {'data': 'info', 'vehicleTypes': 'privateCar'}
url_param_info = api_url + '?' + urllib.parse.urlencode(param_info)
raw_data_info = urllib.request.urlopen(url_param_info).read().decode()
js_result_info = json.loads(raw_data_info)

In [123]:
#js_result_info['results'][3]

#**II) Loading the json extracts into pandas dataframe**

With the following flow:
- The following columns are needed:
 - park_Id
 - name
 - displayAddress
 - district
 - latitude
 - longitude
 - opening_status
 - facilities
 - paymentMethods
 - modifiedDate
- IIa: Write a function for getting URL of carpark image.
 - and insert new column called "url_image"
- IIb: Write a python script for scrapping the dates of public holidays.
- IIc: Write a function for getting carpark today information, such as:
 - Carpark opening time of today: "periodStart"
 - Carpark closing time of today: "periodEnd"
 - Carpark Hourly Charge: "price"
 - Carpark Total Parking Space: "space"  
- IId: Combining all the stuffs, writing python script for outputting a pandas DataFrame.

In [124]:
df1 = pd.DataFrame(js_result_info['results'])[[str('park_Id'), 'name', 'displayAddress', 'district', 'latitude', 'longitude', 'opening_status', 'facilities', 'paymentMethods', 'modifiedDate']]
df1

,park_Id,name,displayAddress,district,latitude,longitude,opening_status,facilities,paymentMethods,modifiedDate
0,10,Kai Tak Cruise Terminal Car Park 1,"1st floor, Kai Tak Cruise Terminal, 33 Shing F...",Kwun Tong District,22.306205,114.213095,OPEN,"[disabilities, evCharger]","[octopus, visa]",2024-11-01 08:29:22
1,12,Amoy Plaza,"77 Ngau Tau Kok Raod, Kowloon Bay, KLN",Kwun Tong District,22.324701,114.216753,OPEN,"[evCharger, disabilities, unloading, washing]","[cash, octopus]",2024-07-12 21:55:17
2,30,Telford Plaza D Carpark,"33 Wai Yip Street, Kowloon Bay, KLN",Kwun Tong District,22.322289,114.212043,OPEN,[evCharger],"[octopus, visa, master]",2025-05-27 12:14:21
3,31,Telford Plaza I Carpark,"33 Wai Yip Street, Kowloon Bay, KLN",Kwun Tong District,22.322686,114.213535,OPEN,"[evCharger, disabilities]","[octopus, visa, master]",2018-01-31 17:19:10
4,32,Telford Plaza II Carpark,"33 Wai Yip Street, Kowloon Bay, KLN",Kwun Tong District,22.321356,114.213266,OPEN,"[evCharger, disabilities]","[octopus, visa, master]",2018-01-31 17:19:44
...,...,...,...,...,...,...,...,...,...,...
469,tdc168p1,The Twins Carpark Tower I,"12 Concorde Road, Kai Tak, San Po Kong, Kowloon",Kowloon City,22.332675,114.200843,CLOSED,NaN,NaN,NaN
470,tdstt100,Hoi Wah Road (II) Car Park,"Hoi Wah Road, Area 16, Tuen Mun, New Territories",Tuen Mun,22.381421,113.973929,CLOSED,NaN,NaN,NaN
471,tdc177p1,KTR350 Car Park,"350 Kwun Tong Road, Kwun Tong, Kowloon",Kwun Tong,22.315039,114.218000,CLOSED,NaN,NaN,NaN
472,tdc248p1,Ka Fu Fong Public Car Park,"186 San Wan Road, Sheung Shui, New Territories",North,22.505996,114.123469,CLOSED,NaN,NaN,NaN


### **IIa) Write a function: Get URL of image of each carpark**
We search anyone URL in the dictionaries 'renditionUrls', with the format:

```
'renditionUrls': {'square': 'https://sps-opendata.pilotsmartke.gov.hk/rest/getRendition/fs-1%3A693265207413252869411532657339312395903827562313.JPG/square.png',
'thumbnail': 'https://sps-opendata.pilotsmartke.gov.hk/rest/getRendition/fs-1%3A693265207413252869411532657339312395903827562313.JPG/thumbnail.png',
'banner': 'https://sps-opendata.pilotsmartke.gov.hk/rest/getRendition/fs-1%3A693265207413252869411532657339312395903827562313.JPG/banner.png'}
```

<u>**Write a function "get_url_image"**</u> with
- input as "js_row", the json item for a single carpark
- return output as the first URL item in the dictionary

Besides, in the function you need
- If dictionary 'renditionUrls' cannot be found, return None.
- If 'renditionUrls' is found but not the URL, return None as well.

---



In [125]:
def get_url_image(js_row):
  url_image = []
  for x in js_row:
    if 'renditionUrls' in x:
      if 'square' in x['renditionUrls']:
        url_image.append(x['renditionUrls']['square'])
      elif 'carpark_photo' in x['renditionUrls']:
        url_image.append(x['renditionUrls']['carpark_photo'])
    else:
      url_image.append(None)
  return url_image

get_url_image(js_result_info['results'])

#js_row = js_result_info['results']
#print(js_row[0]['renditionUrls']['square'])

['https://sps-opendata.pilotsmartke.gov.hk/rest/getRendition/fs-1%3A693265207413252869411532657339312395903827562313.JPG/square.png',
 None,
 None,
 None,
 None,
 'https://sps-opendata.pilotsmartke.gov.hk/rest/getRendition/fs-1%3A859280094011967658179347734190846999967397591797.jpg/square.png',
 None,
 'https://sps-opendata.pilotsmartke.gov.hk/rest/getRendition/fs-1%3A962030815429880907653104672944223004044154159758.JPG/square.png',
 None,
 'https://sps-opendata.pilotsmartke.gov.hk/rest/getRendition/fs-1%3A1008505939494327513640339758408113224686274981686.jpg/square.png',
 None,
 'https://sps-opendata.pilotsmartke.gov.hk/rest/getRendition/fs-1%3A642584489794480030606579782864901108463497529270.JPG/square.png',
 'https://sps-opendata.pilotsmartke.gov.hk/rest/getRendition/fs-1%3A469807454887318353643252933707115157023274119448.jpg/square.png',
 None,
 None,
 'https://sps-opendata.pilotsmartke.gov.hk/rest/getRendition/fs-1%3A1024777083027213348588968133748246333733318794266.png/square.png

In [126]:
df2 = df1.assign(url_image=get_url_image(js_result_info['results']))
df2

,park_Id,name,displayAddress,district,latitude,longitude,opening_status,facilities,paymentMethods,modifiedDate,url_image
0,10,Kai Tak Cruise Terminal Car Park 1,"1st floor, Kai Tak Cruise Terminal, 33 Shing F...",Kwun Tong District,22.306205,114.213095,OPEN,"[disabilities, evCharger]","[octopus, visa]",2024-11-01 08:29:22,https://sps-opendata.pilotsmartke.gov.hk/rest/...
1,12,Amoy Plaza,"77 Ngau Tau Kok Raod, Kowloon Bay, KLN",Kwun Tong District,22.324701,114.216753,OPEN,"[evCharger, disabilities, unloading, washing]","[cash, octopus]",2024-07-12 21:55:17,None
2,30,Telford Plaza D Carpark,"33 Wai Yip Street, Kowloon Bay, KLN",Kwun Tong District,22.322289,114.212043,OPEN,[evCharger],"[octopus, visa, master]",2025-05-27 12:14:21,None
3,31,Telford Plaza I Carpark,"33 Wai Yip Street, Kowloon Bay, KLN",Kwun Tong District,22.322686,114.213535,OPEN,"[evCharger, disabilities]","[octopus, visa, master]",2018-01-31 17:19:10,None
4,32,Telford Plaza II Carpark,"33 Wai Yip Street, Kowloon Bay, KLN",Kwun Tong District,22.321356,114.213266,OPEN,"[evCharger, disabilities]","[octopus, visa, master]",2018-01-31 17:19:44,None
...,...,...,...,...,...,...,...,...,...,...,...
469,tdc168p1,The Twins Carpark Tower I,"12 Concorde Road, Kai Tak, San Po Kong, Kowloon",Kowloon City,22.332675,114.200843,CLOSED,NaN,NaN,NaN,http://resource.data.one.gov.hk/td/carpark/tdc...
470,tdstt100,Hoi Wah Road (II) Car Park,"Hoi Wah Road, Area 16, Tuen Mun, New Territories",Tuen Mun,22.381421,113.973929,CLOSED,NaN,NaN,NaN,http://resource.data.one.gov.hk/td/carpark/tds...
471,tdc177p1,KTR350 Car Park,"350 Kwun Tong Road, Kwun Tong, Kowloon",Kwun Tong,22.315039,114.218000,CLOSED,NaN,NaN,NaN,http://resource.data.one.gov.hk/td/carpark/tdc...
472,tdc248p1,Ka Fu Fong Public Car Park,"186 San Wan Road, Sheung Shui, New Territories",North,22.505996,114.123469,CLOSED,NaN,NaN,NaN,http://resource.data.one.gov.hk/td/carpark/tdc...


### **IIb) Write a script: Get Public Holidays in Hong Kong**

<u>**Write a script for webscrapping with package BeautifulSoup**</u> through the website: [General holidays for 2025](https://www.gov.hk/en/about/abouthk/holiday/2025.htm)

save those dates in format 'YYYY-mm-dd' packed with a list in the variable 'ph_dates', as following:

```
# ph_dates

['2024-01-01',
 '2024-02-10',
 '2024-02-12',
 '2024-02-13',
 '2024-03-29',
 '2024-03-30',
 '2024-04-01',
 '2024-04-04',
 '2024-05-01',
 '2024-05-15',
 '2024-06-10',
 '2024-07-01',
 '2024-09-18',
 '2024-10-01',
 '2024-10-11',
 '2024-12-25',
 '2024-12-26']
```



In [127]:
# ph_dates
x = requests.get('https://www.gov.hk/en/about/abouthk/holiday/2025.htm')
y = bsoup(x.content, 'html.parser')
b = y.find_all('td', class_='date')

date_list = []
for c in b:
  date_list.append(c.text.replace('\xa0', ' ').split(' '))
date_list = date_list[1:]
#print(date_list)

ph_dates = []
month_dict = {'January': '01-', 'February': '02-', 'March': '03-', 'April': '04-', 'May': '05-', 'June': '06-', 'July': '07-', 'August': '08-', 'September': '09-', 'October': '10-', 'November': '11-', 'December': '12-'}
for i in date_list:
  x = '2025-' + month_dict[i[1]]
  if len(i[0]) == 1:
    x += '0' + i[0]
  else:
    x += i[0]
  ph_dates.append(x)
#print(date_list_final)

print(('[' + str(np.c_[ph_dates]).replace('[','').replace(']',','))[:-2] + ']')
#print(ph_dates)

['2025-01-01',
 '2025-01-29',
 '2025-01-30',
 '2025-01-31',
 '2025-04-04',
 '2025-04-18',
 '2025-04-19',
 '2025-04-21',
 '2025-05-01',
 '2025-05-05',
 '2025-05-31',
 '2025-07-01',
 '2025-10-01',
 '2025-10-07',
 '2025-10-29',
 '2025-12-25',
 '2025-12-26']


### **IIc) Write a function: Get information for each carpark, according to the date inputted.**

The hours opening/closing, hourly charge, and parking spaces can be found in dictionaries 'privateCar' and nested dictionary 'hourlyCharges', such as the following example:

```
 'privateCar': {'hourlyCharges': [{'weekdays': ['MON',
     'TUE',
     'WED',
     'THU',
     'FRI'],
    'excludePublicHoliday': True,
    'periodStart': '07:00',
    'periodEnd': '23:00',
    'price': 15,
    'type': 'hourly',
    'covered': 'covered',
    'usageMinimum': 1,
    'remark': ''},
   {'weekdays': ['SAT', 'SUN', 'PH'],
    'excludePublicHoliday': False,
    'periodStart': '07:00',
    'periodEnd': '23:00',
    'price': 20,
    'usageMinimum': 1,
    'type': 'hourly',
    'covered': 'covered',
    'remark': ''}],
  'spaceUNL': 0,
  'spaceEV': 0,
  'spaceDIS': 0,
  'space': 112},
```

<u>**Write a function "get_todayinfo"**</u> with
- input as
 - "js_row", the json item for a single carpark
 - "ph_dates", the list of dates in string of public holidays
 - "today_date_str", the date in string with format "YYYY-mm-dd"
- return a dictionary with
 - "periodStart", time when carpark is opened
 - "periodEnd", time when carpark is closed
 - "price", hourly charge
 - "space", no. of parking spaces in the carpark.

Besides, in the function you need
- Set to None if "periodStart", "periodEnd", or "price" have not been mentioned in data
- Set to 0 if "space" has not been mentioned in data.
- default value of parameter "today_date_str" is assigned to be None. Then today date is assigned in the function with variable "today_date_str"

CAUTION: For some carparks, the keys of some dictionaries may be missed. Some default value need to be set once you cannot call ite in the dictionary.

---

Expected Outputs:

```
# get_todayinfo(js_row, ph_dates)

{'periodStart': '07:00',
 'periodEnd': '23:00',
 'price': 15,
 'space': 112,
 'today': '2024-10-18',
 'today_weekday': 'FRI'}

# get_todayinfo(js_row, ph_dates, '2024-12-25')

{'periodStart': '07:00',
 'periodEnd': '23:00',
 'price': 20,
 'space': 112,
 'today': '2024-12-25',
 'today_weekday': 'PH'}
```




In [128]:
from datetime import date
from datetime import datetime

def get_todayinfo(js_row, ph_dates, today_date_str=str(date.today())):
  list_todayinfo = []
  days_list = ['MON', 'TUE', 'WED', 'THU', 'FRI', 'SAT', 'SUN']
  for x in js_row:
    day_of_week = days_list[datetime.strptime(today_date_str, "%Y-%m-%d").weekday()]
    dict_todayinfo = {}
    if 'privateCar' in x:
      if 'hourlyCharges' in x['privateCar']:
        ph_count = 0
        for z in x['privateCar']['hourlyCharges']:
          if 'PH' in z['weekdays']:
            ph_count += 1
        if ph_count > 0 and today_date_str in ph_dates:
          day_of_week = 'PH'
        for y in x['privateCar']['hourlyCharges']:
          if day_of_week in y['weekdays']:
            if 'periodStart' in y:
              dict_todayinfo.update({'periodStart': y['periodStart']})
            else:
              dict_todayinfo.update({'periodStart': None})
            if 'periodEnd' in y:
              dict_todayinfo.update({'periodEnd': y['periodEnd']})
            else:
              dict_todayinfo.update({'periodEnd': None})
            if 'price' in y:
              dict_todayinfo.update({'price': y['price']})
            else:
              dict_todayinfo.update({'price': None})
      if 'hourlyCharges' not in x['privateCar']:
        dict_todayinfo.update({'periodStart': None, 'periodEnd': None, 'price': None}) #, 'space': 0})
      if 'space' in x['privateCar']:
        dict_todayinfo.update({'space': x['privateCar']['space']})
      if 'space' not in x['privateCar']:
        dict_todayinfo.update({'space': 0})
    else:
      dict_todayinfo.update({'periodStart': None, 'periodEnd': None, 'price': None, 'space': 0})
    dict_todayinfo.update({'today': today_date_str, 'today_weekday': day_of_week})
    list_todayinfo.append(dict_todayinfo)
  return list_todayinfo

In [129]:
input_date = str(input('What date (YYYY-MM-DD) would you like to check the vacancies for? [Leaving this field blank defaults to today]: '))
if input_date == '':
  input_date = str(date.today())
print(('[' + str(np.c_[get_todayinfo(js_result_info['results'], ph_dates, input_date)]).replace('[','').replace(']',','))[:-2] + ']')
#print(get_todayinfo(js_result_info['results'], ph_dates, '2024-12-25'))

What date (YYYY-MM-DD) would you like to check the vacancies for? [Leaving this field blank defaults to today]: 2025-05-01
[{'periodStart': '07:00', 'periodEnd': '23:00', 'price': 20, 'space': 112, 'today': '2025-05-01', 'today_weekday': 'PH'},
 {'periodStart': '00:00', 'periodEnd': '24:00', 'price': 25, 'space': 389, 'today': '2025-05-01', 'today_weekday': 'PH'},
 {'periodStart': '00:00', 'periodEnd': '24:00', 'price': 22, 'space': 70, 'today': '2025-05-01', 'today_weekday': 'PH'},
 {'periodStart': '00:00', 'periodEnd': '24:00', 'price': 22, 'space': 170, 'today': '2025-05-01', 'today_weekday': 'PH'},
 {'periodStart': '00:00', 'periodEnd': '24:00', 'price': 22, 'space': 94, 'today': '2025-05-01', 'today_weekday': 'PH'},
 {'periodStart': '00:00', 'periodEnd': '24:00', 'price': 20, 'space': 30, 'today': '2025-05-01', 'today_weekday': 'PH'},
 {'periodStart': '07:00', 'periodEnd': '23:00', 'price': 20, 'space': 152, 'today': '2025-05-01', 'today_weekday': 'PH'},
 {'periodStart': '07:00', 

In [130]:
#print(date.today())
js_result_info['results'][2]['privateCar']['hourlyCharges'] #['privileges'] #['hourlyCharges'][0]['weekdays']
#'FRI' in js_result_info['results'][0]['privateCar']['hourlyCharges'][0]['weekdays']

#today_date_str = '2025-12-25'
#print((datetime.strptime(str(today_date_str), "%Y-%m-%d")).weekday())

[{'weekdays': ['MON', 'TUE', 'WED', 'THU', 'FRI', 'SAT', 'SUN', 'PH'],
  'excludePublicHoliday': False,
  'covered': 'covered',
  'type': 'hourly',
  'remark': '',
  'price': 22,
  'periodEnd': '24:00',
  'periodStart': '00:00'}]

### **IId) Putting all the things together - Output the pandas data frame**

<u>**Write a script to get the full dataset merged**</u>.


Save the dataset with name "df_carpark".

In [131]:
# df_carpark
df_carpark = df2.join(pd.DataFrame(get_todayinfo(js_result_info['results'], ph_dates, input_date)))
df_carpark_today = df2.join(pd.DataFrame(get_todayinfo(js_result_info['results'], ph_dates)))
df_carpark

,park_Id,name,displayAddress,district,latitude,longitude,opening_status,facilities,paymentMethods,modifiedDate,url_image,periodStart,periodEnd,price,space,today,today_weekday
0,10,Kai Tak Cruise Terminal Car Park 1,"1st floor, Kai Tak Cruise Terminal, 33 Shing F...",Kwun Tong District,22.306205,114.213095,OPEN,"[disabilities, evCharger]","[octopus, visa]",2024-11-01 08:29:22,https://sps-opendata.pilotsmartke.gov.hk/rest/...,07:00,23:00,20.0,112,2025-05-01,PH
1,12,Amoy Plaza,"77 Ngau Tau Kok Raod, Kowloon Bay, KLN",Kwun Tong District,22.324701,114.216753,OPEN,"[evCharger, disabilities, unloading, washing]","[cash, octopus]",2024-07-12 21:55:17,None,00:00,24:00,25.0,389,2025-05-01,PH
2,30,Telford Plaza D Carpark,"33 Wai Yip Street, Kowloon Bay, KLN",Kwun Tong District,22.322289,114.212043,OPEN,[evCharger],"[octopus, visa, master]",2025-05-27 12:14:21,None,00:00,24:00,22.0,70,2025-05-01,PH
3,31,Telford Plaza I Carpark,"33 Wai Yip Street, Kowloon Bay, KLN",Kwun Tong District,22.322686,114.213535,OPEN,"[evCharger, disabilities]","[octopus, visa, master]",2018-01-31 17:19:10,None,00:00,24:00,22.0,170,2025-05-01,PH
4,32,Telford Plaza II Carpark,"33 Wai Yip Street, Kowloon Bay, KLN",Kwun Tong District,22.321356,114.213266,OPEN,"[evCharger, disabilities]","[octopus, visa, master]",2018-01-31 17:19:44,None,00:00,24:00,22.0,94,2025-05-01,PH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,tdc168p1,The Twins Carpark Tower I,"12 Concorde Road, Kai Tak, San Po Kong, Kowloon",Kowloon City,22.332675,114.200843,CLOSED,NaN,NaN,NaN,http://resource.data.one.gov.hk/td/carpark/tdc...,None,None,NaN,0,2025-05-01,THU
470,tdstt100,Hoi Wah Road (II) Car Park,"Hoi Wah Road, Area 16, Tuen Mun, New Territories",Tuen Mun,22.381421,113.973929,CLOSED,NaN,NaN,NaN,http://resource.data.one.gov.hk/td/carpark/tds...,None,None,NaN,0,2025-05-01,THU
471,tdc177p1,KTR350 Car Park,"350 Kwun Tong Road, Kwun Tong, Kowloon",Kwun Tong,22.315039,114.218000,CLOSED,NaN,NaN,NaN,http://resource.data.one.gov.hk/td/carpark/tdc...,None,None,NaN,0,2025-05-01,THU
472,tdc248p1,Ka Fu Fong Public Car Park,"186 San Wan Road, Sheung Shui, New Territories",North,22.505996,114.123469,CLOSED,NaN,NaN,NaN,http://resource.data.one.gov.hk/td/carpark/tdc...,None,None,NaN,0,2025-05-01,THU


# **III) Get Real-time Vacancy for each carpark**
Besides the carpark information, the API can also be called for the real-time vacancies.

Please re-check the following:
- The API page is here. ([In Traditional Chinese](https://data.gov.hk/tc-data/dataset/hk-dpo-datagovhk1-carpark-info-vacancy/resource/f4c792c6-071c-4a64-888b-afeea33d5ad7), [In English](https://data.gov.hk/en-data/dataset/hk-dpo-datagovhk1-carpark-info-vacancy/resource/01752c62-a6b6-4ddc-bf2d-25efccadc143))
- and the data dictionary is [here](https://resource.data.one.gov.hk/opendata/carpark/Parking_Vacancy_Data_Specification.pdf).

<u>**Write a script for obtaining the full pandas data frame**</u>, with the following instructions:
- In calling API, set the "data" parameter to be "vacancy"
- set the "vehicleType" parameter to be "privateCar" as well
- 3 columns are in dataset
 - "park_Id", for joining the carpark info dataset.
 - "vacancy", the no. of available carpark spaces
 - "lastupdate", Last Update time of the "vacancy" number.
- Only "vacancy_type" = "A", the vacancy no. will be taken into account.
- In default, set "vacancy" as 0, and "lastupdate" as None, if the carpark does not have this information.
- Name the dataset as "df_vacancy".

---

Expected JSON for a single carpark

```
# js_result[0]

{'park_Id': '10',
 'privateCar': [{'vacancy_type': 'A',
   'vacancy': 29,
   'lastupdate': '2024-10-18 16:04:59'}]}
```

Note the vacancy dataset is has the same number of rows as the carpark info dataset.




In [132]:
# df_vacancy
vacancy_list = []
for x in js_result_vacancy['results']:
  vacancy_dict = {'park_Id': str(x['park_Id'])}
  if 'privateCar' in x:
    if x['privateCar'][0]['vacancy_type'] == 'A':
      vacancy_dict.update({'vacancy': x['privateCar'][0]['vacancy']})
    else:
      vacancy_dict.update({'vacancy': 0})
    vacancy_dict.update({'lastupdate': x['privateCar'][0]['lastupdate']})
  else:
    vacancy_dict.update({'vacancy': 0, 'lastupdate': None})
  vacancy_list.append(vacancy_dict)

#print(vacancy_list)
print(('[' + str(np.c_[vacancy_list]).replace('[','').replace(']',','))[:-2] + ']')

[{'park_Id': '10', 'vacancy': 105, 'lastupdate': '2025-06-21 20:43:51'},
 {'park_Id': '12', 'vacancy': 26, 'lastupdate': '2025-06-20 09:30:24'},
 {'park_Id': '30', 'vacancy': 11, 'lastupdate': '2025-06-21 20:41:30'},
 {'park_Id': '31', 'vacancy': 0, 'lastupdate': '2025-06-21 19:28:00'},
 {'park_Id': '32', 'vacancy': 0, 'lastupdate': '2025-06-21 20:37:00'},
 {'park_Id': '37', 'vacancy': 42, 'lastupdate': '2025-06-21 20:20:37'},
 {'park_Id': '38', 'vacancy': 128, 'lastupdate': '2025-06-20 23:14:37'},
 {'park_Id': '59', 'vacancy': 30, 'lastupdate': '2025-06-21 20:43:51'},
 {'park_Id': '74', 'vacancy': 0, 'lastupdate': '2025-06-21 03:01:17'},
 {'park_Id': '75', 'vacancy': 48, 'lastupdate': '2025-06-21 20:43:55'},
 {'park_Id': '77', 'vacancy': 85, 'lastupdate': '2025-06-21 20:29:20'},
 {'park_Id': '78', 'vacancy': 13, 'lastupdate': '2025-06-17 18:33:29'},
 {'park_Id': '81', 'vacancy': 72, 'lastupdate': '2025-06-20 09:11:31'},
 {'park_Id': '88', 'vacancy': 69, 'lastupdate': '2025-06-20 14:48

In [133]:
df_vacancy = pd.DataFrame(vacancy_list)
df_vacancy

,park_Id,vacancy,lastupdate
0,10,105,2025-06-21 20:43:51
1,12,26,2025-06-20 09:30:24
2,30,11,2025-06-21 20:41:30
3,31,0,2025-06-21 19:28:00
4,32,0,2025-06-21 20:37:00
...,...,...,...
469,tdc168p1,154,2025-06-21 20:47:36
470,tdstt100,2,2025-06-21 20:47:15
471,tdc177p1,47,2025-06-21 20:47:51
472,tdc248p1,22,2025-06-21 20:47:43


In [134]:
#js_result_vacancy['results'][0]

# **IV) Join both datasets**

Finally, <u>**write a script to join df_carpark and df_vacancy**</u>, with the following instruction.
- Applying "Left-Join"
- on the common column "park_Id"
- saving as "df_out"

---




In [135]:
df_out = df_carpark_today.join(df_vacancy.iloc[:,1:])
df_out

,park_Id,name,displayAddress,district,latitude,longitude,opening_status,facilities,paymentMethods,modifiedDate,url_image,periodStart,periodEnd,price,space,today,today_weekday,vacancy,lastupdate
0,10,Kai Tak Cruise Terminal Car Park 1,"1st floor, Kai Tak Cruise Terminal, 33 Shing F...",Kwun Tong District,22.306205,114.213095,OPEN,"[disabilities, evCharger]","[octopus, visa]",2024-11-01 08:29:22,https://sps-opendata.pilotsmartke.gov.hk/rest/...,07:00,23:00,20.0,112,2025-06-21,SAT,105,2025-06-21 20:43:51
1,12,Amoy Plaza,"77 Ngau Tau Kok Raod, Kowloon Bay, KLN",Kwun Tong District,22.324701,114.216753,OPEN,"[evCharger, disabilities, unloading, washing]","[cash, octopus]",2024-07-12 21:55:17,None,00:00,24:00,25.0,389,2025-06-21,SAT,26,2025-06-20 09:30:24
2,30,Telford Plaza D Carpark,"33 Wai Yip Street, Kowloon Bay, KLN",Kwun Tong District,22.322289,114.212043,OPEN,[evCharger],"[octopus, visa, master]",2025-05-27 12:14:21,None,00:00,24:00,22.0,70,2025-06-21,SAT,11,2025-06-21 20:41:30
3,31,Telford Plaza I Carpark,"33 Wai Yip Street, Kowloon Bay, KLN",Kwun Tong District,22.322686,114.213535,OPEN,"[evCharger, disabilities]","[octopus, visa, master]",2018-01-31 17:19:10,None,00:00,24:00,22.0,170,2025-06-21,SAT,0,2025-06-21 19:28:00
4,32,Telford Plaza II Carpark,"33 Wai Yip Street, Kowloon Bay, KLN",Kwun Tong District,22.321356,114.213266,OPEN,"[evCharger, disabilities]","[octopus, visa, master]",2018-01-31 17:19:44,None,00:00,24:00,22.0,94,2025-06-21,SAT,0,2025-06-21 20:37:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,tdc168p1,The Twins Carpark Tower I,"12 Concorde Road, Kai Tak, San Po Kong, Kowloon",Kowloon City,22.332675,114.200843,CLOSED,NaN,NaN,NaN,http://resource.data.one.gov.hk/td/carpark/tdc...,None,None,NaN,0,2025-06-21,SAT,154,2025-06-21 20:47:36
470,tdstt100,Hoi Wah Road (II) Car Park,"Hoi Wah Road, Area 16, Tuen Mun, New Territories",Tuen Mun,22.381421,113.973929,CLOSED,NaN,NaN,NaN,http://resource.data.one.gov.hk/td/carpark/tds...,None,None,NaN,0,2025-06-21,SAT,2,2025-06-21 20:47:15
471,tdc177p1,KTR350 Car Park,"350 Kwun Tong Road, Kwun Tong, Kowloon",Kwun Tong,22.315039,114.218000,CLOSED,NaN,NaN,NaN,http://resource.data.one.gov.hk/td/carpark/tdc...,None,None,NaN,0,2025-06-21,SAT,47,2025-06-21 20:47:51
472,tdc248p1,Ka Fu Fong Public Car Park,"186 San Wan Road, Sheung Shui, New Territories",North,22.505996,114.123469,CLOSED,NaN,NaN,NaN,http://resource.data.one.gov.hk/td/carpark/tdc...,None,None,NaN,0,2025-06-21,SAT,22,2025-06-21 20:47:43


# **V) User location string input fuzzy match optimization**

The user input should fuzzymatch to string portions in the `['name', 'displayAddress', 'district']` columns


In [136]:
################################
#### for score optimization ####
################################

#print(len(df_carpark[df_carpark['displayAddress'].str.contains(r'Wai Yip', case=False, na=False)]))
#df_carpark[df_carpark['displayAddress'].str.contains(r'Wai Yip', case=False, na=False)]

In [137]:
def normalize_text(text):
    """Normalize text but preserve word order"""
    if pd.isna(text):
        return ""
    return str(text).lower().replace(",", " ")  # Keep spaces for word order

def fuzzy_search_carparks(df, search_string):
    """
    Strict fuzzy search that:
    - Respects word order ("Tak Kai" ≠ "Kai Tak")
    - Matches partial strings ("kaitak" → "Kai Tak")
    - Returns original DataFrame rows
    """
    search_norm = normalize_text(search_string)

    # Normalize columns (preserving word order)
    df['norm_name'] = df['name'].apply(normalize_text)
    df['norm_address'] = df['displayAddress'].apply(normalize_text)
    df['norm_district'] = df['district'].apply(normalize_text)

    def row_has_match(row):
        # Use partial_ratio to respect word order
        name_score = fuzz.partial_ratio(search_norm, row['norm_name'])
        addr_score = fuzz.partial_ratio(search_norm, row['norm_address'])
        dist_score = fuzz.partial_ratio(search_norm, row['norm_district'])

        return (name_score > 84) or (addr_score > 78) or (dist_score > 87) # name_score > 84; addr_score > 78; dist_score > 87 ==> after manually tweaking the values, these seem to give the best results

    matches = df[df.apply(row_has_match, axis=1)].copy()
    return matches.drop(columns=['norm_name', 'norm_address', 'norm_district'])

# **VI) Final results**

<u>**Maximum capacity of carpark of chosen date and location**</u>

In [142]:
input_loc = str(input("Please input the location of where you would like to check parking space (e.g. malls, streets, districts, etc): "))
fuzzy_search_carparks(df_carpark, input_loc)

Please input the location of where you would like to check parking space (e.g. malls, streets, districts, etc) [leave blank to return all]: 


<u>**Real-time vacancy of carpark of chosen location**</u>

In [141]:
fuzzy_search_carparks(df_out, input_loc)

,park_Id,name,displayAddress,district,latitude,longitude,opening_status,facilities,paymentMethods,modifiedDate,url_image,periodStart,periodEnd,price,space,today,today_weekday,vacancy,lastupdate
225,tdc48p1,Lai Chi Kok Park,"1 Lai Wan Road, Lai Chi Kok, Kowloon",Sham Shui Po,22.339746,114.138222,OPEN,NaN,NaN,NaN,http://resource.data.one.gov.hk/td/carpark/tdc...,None,None,NaN,0,2025-06-21,SAT,1,2025-06-21 20:47:07
277,tdc14p5406,Wah Lai Car Park,"Wah Lai Estate Car Park, 282 Lai King Hill Roa...",Kwai Tsing,22.342508,114.137718,OPEN,NaN,NaN,NaN,http://resource.data.one.gov.hk/td/carpark/tdc...,None,None,NaN,0,2025-06-21,SAT,0,2025-06-21 20:47:08


---


#**FUTURE WORKS**

I want to make something like this in Tableau dashboard.

![Example Image](https://drive.google.com/uc?id=1-I14TdnpkGwwObWs1ZNGt2Ufb2NMIBSH)